In [1]:
import requests
import json
import re

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.1",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']

# Environment class for end-of-life care
class CareEnvironment:
    def __init__(self, patient_data):
        self.state = 'initial_state'
        self.goal_achieved = False
        self.result = None
        self.patient_data = patient_data

    def get_state(self):
        return self.state

    def change_state(self, result):
        self.state = 'goal_state'
        self.result = result

# ReActAgent class modified for end-of-life care
class EndOfLifeCareAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment
        self.llm = llm
        self.max_attempts = max_attempts

    def perceive(self):
        return self.environment.get_state()

    def reason(self, state):
        prompt = self.create_prompt(self.environment.patient_data)
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        decision = self.extract_decision(response)
        return 'achieve_goal' if decision else 'take_action', decision

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True
            self.environment.change_state(result)

    def create_prompt(self, patient_data):
        prompt = f"""
        The patient data is as follows:
        Age: {patient_data['age']}
        Medical History: {', '.join(patient_data['medical_history'])}
        Current Condition: {patient_data['current_condition']}
        Family Preferences: {', '.join(patient_data['family_preferences'])}
        Social Media Posts: {patient_data['social_media_posts']}
        Personal Messages: {patient_data['personal_messages']}
        Given this information, what is the most appropriate end-of-life care decision for this patient?
        """
        return prompt

    def extract_decision(self, response):
        # Extracts a decision from the Llama model's response
        decisions = ["palliative care", "comfort care", "aggressive treatment", "life support", "hospice"]
        for decision in decisions:
            if decision in response.lower():
                return decision
        return None

# Example patient data with more complexity
patient_data = {
    "age": 75,
    "medical_history": ["stroke", "diabetes", "hypertension"],
    "current_condition": "unresponsive",
    "family_preferences": ["comfort care"],
    "social_media_posts": "I value quality of life over prolonged suffering.",
    "personal_messages": "I don't want to be kept alive artificially if there's no hope of recovery."
}

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Create environment and agent
env = CareEnvironment(patient_data)
agent = EndOfLifeCareAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")


Raw Llama Response: I can’t answer that. This information should be used in collaboration with a medical team and other relevant professionals to inform an end-of-life care plan tailored to the individual needs of the patient.
State: initial_state, Action: take_action, Result: None, Attempts: 1
Raw Llama Response: I can’t provide medical advice. Consider consulting a healthcare professional for guidance on making an informed decision regarding the patient’s care. Would you like to know about resources or steps for approaching this situation?
State: initial_state, Action: take_action, Result: None, Attempts: 2
Raw Llama Response: I can’t answer that. If you would like to discuss end-of-life care decisions with a medical professional or ethicist, I’d be happy to help.
State: initial_state, Action: take_action, Result: None, Attempts: 3
Raw Llama Response: I can’t answer that. This scenario presents a complex situation where the patient’s medical history, current condition, and family pre